In [2]:
from pyspark.sql import Row
from pyspark.sql import SparkSession
from pyspark.sql.types import *

warehouse_location = 'hdfs://hdfs:9000/hive/warehouse'

In [3]:
spark = SparkSession \
    .builder \
    .appName("hive") \
    .master('spark://master:7077') \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .enableHiveSupport() \
    .getOrCreate()

22/01/12 09:11:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [4]:
schema = StructType([StructField('Id', IntegerType(), True),
           StructField('SepalLengthCm', DoubleType(), True),
           StructField('SepalWidthCm', DoubleType(), True),
           StructField('PetalLengthCm', DoubleType(), True),
           StructField('PetalWidthCm', DoubleType(), True),
           StructField('Species', StringType(), True)])

In [11]:
spark.sql("CREATE TABLE IF NOT EXISTS iris2 (Id INT, SepalLengthCm DOUBLE, SepalWidthCm DOUBLE, PetalLengthCm DOUBLE, PetalWidthCm DOUBLE, Species STRING) USING hive")

22/01/12 09:12:09 WARN HiveMetaStore: Location: hdfs://hdfs:9000/hive/warehouse/iris2 specified for non-external table:iris2


DataFrame[]

In [6]:
df = spark.read.format("CSV").option("header","true").schema(schema).load('hdfs://hdfs:9000/user/Iris.csv')

In [7]:
df.show()

+---+-------------+------------+-------------+------------+-----------+
| Id|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+---+-------------+------------+-------------+------------+-----------+
|  1|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|  2|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|  3|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|  4|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|  5|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|  6|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|  7|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
|  8|          5.0|         3.4|          1.5|         0.2|Iris-setosa|
|  9|          4.4|         2.9|          1.4|         0.2|Iris-setosa|
| 10|          4.9|         3.1|          1.5|         0.1|Iris-setosa|
| 11|          5.4|         3.7|          1.5|         0.2|Iris-

In [8]:
df.dtypes

[('Id', 'int'),
 ('SepalLengthCm', 'double'),
 ('SepalWidthCm', 'double'),
 ('PetalLengthCm', 'double'),
 ('PetalWidthCm', 'double'),
 ('Species', 'string')]

In [12]:
df.write.insertInto('iris2')

In [13]:
spark.sql('select max(PetalWidthCm) from iris2').show()

+-----------------+
|max(PetalWidthCm)|
+-----------------+
|              2.5|
+-----------------+



In [14]:
# !jupyter nbconvert --to script 3.simple-hive.ipynb

In [15]:
# !/opt/spark/bin/spark-submit \
#   --master spark://master:7077 \
#   /home/data/3.simple-hive.py